In [1]:
import path_configs # noqa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import settings
from datetime import datetime
import os
settings.init()

from modules.ClassifierGenerators import * # noqa
from modules.EncoderGenerators import * # noqa

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# assumes that array is not zero
def scaled(tensor):
    return (tensor-tf.math.reduce_min(tensor))/(tf.math.reduce_max(tensor)-tf.
                                                math.reduce_min(tensor))

AttributeError: module 'tensorflow.python.keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [8]:
for x, y1, y2 in TrainClassifierGenerator():
    print(x.shape)
    print("range: {} to {}".format(tf.math.reduce_min(x), tf.math.reduce_max(x)))
    print(y1)
    print(y2)
    break

(256, 128)
range: 0.0 to 1.0
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], shape=(256,), dtype=int32)
tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1

In [4]:
def base_model(main_model_input):
    base_layer = tf.keras.layers.RepeatVector(settings.AE_TIMESTEPS)(main_model_input)
    base_layer = tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True)(base_layer)
    base_layer = tf.keras.layers.LSTM(64, activation='tanh', return_sequences=False)(base_layer)
    return base_layer

# age branch
def build_age_branch(inputs, num_ages = 9):    
    age_layer = tf.keras.layers.Dense(512, activation='relu')(inputs)
    age_layer = tf.keras.layers.Dropout(0.4)(age_layer)
    age_layer = tf.keras.layers.Dense(256, activation='relu')(age_layer)
    age_layer = tf.keras.layers.Dropout(0.4)(age_layer)
    age_output = tf.keras.layers.Dense(num_ages, activation='softmax', name='age')(age_layer)
    return age_output

# gender branch,
def build_gender_branch(inputs, num_genders = 2):
    gender_layer = tf.keras.layers.Dense(512, activation='relu')(inputs)
    gender_layer = tf.keras.layers.Dropout(0.4)(gender_layer)
    gender_layer = tf.keras.layers.Dense(256, activation='relu')(gender_layer)
    gender_layer = tf.keras.layers.Dropout(0.4)(gender_layer)
    gender_output = tf.keras.layers.Dense(num_genders, activation='linear', name='gender')(gender_layer)
    return gender_output


def assemble_full_model():
    main_model_input = tf.keras.layers.Input(shape=(settings.AE_LATENT_DIM))

    inputs = base_model(main_model_input)

    age_branch = build_age_branch(inputs)
    gender_branch = build_gender_branch(inputs)

    model = tf.keras.models.Model(inputs=main_model_input, outputs=[age_branch, gender_branch], name="voice_net")
    return model

model = assemble_full_model()

In [5]:
model.summary()

Model: "voice_net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 64, 128)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 64, 128)      131584      repeat_vector[0][0]              
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 64)           49408       lstm[0][0]                       
__________________________________________________________________________________________

In [6]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [7]:
# tf.keras.utils.plot_model(model, show_shapes=True)

In [8]:
now = datetime.now()
date_time = now.strftime("%m_%d_%Y_%H_%M")

NAME = ("main_model_" + date_time)

for folder_name in ['logs', 'saved_models', os.path.join('saved_models', 'checkpoints')]:
    try:
        os.mkdir(folder_name)
        print("Directory", folder_name,  "created ")
    except FileExistsError:
        print("Directory", folder_name,  "already exists")

Directory logs already exists
Directory saved_models already exists
Directory saved_models/checkpoints already exists


In [9]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='logs\\{}'.format(NAME)),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join('saved_models', 'checkpoints', '{}.h5'.format(NAME)),
        monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
    ]

In [10]:
model.fit(
    TrainClassifierGenerator().prefetch(tf.data.experimental.AUTOTUNE),
    epochs=settings.MODEL_EPOCHS,
    steps_per_epoch=( settings.TRAIN_DATA_LEN // 2 ) // settings.MODEL_BATCH_SIZE,
    verbose=1,
    validation_data=ValidationClassifierGenerator().prefetch(tf.data.experimental.AUTOTUNE),
    validation_steps=( settings.VAL_DATA_LEN // 2 ) // settings.MODEL_BATCH_SIZE
    # callbacks=callbacks
    )

Epoch 1/10
  65/1816 [>.............................] - ETA: 29:47 - loss: nan - age_loss: 1.0488 - gender_loss: nan - age_accuracy: 0.1430 - gender_accuracy: 0.1667

KeyboardInterrupt: 

In [11]:
model.save(os.path.join('saved_models', 'auto_{}.h5'.format(NAME)))